In [ ]:
import multiprocessing
from multiprocessing import Pool
import pandas as pd
from PKM_multiprocess import ParallelCompute_PKM as PC_PKM

if __name__ == "__main__":
    pool = Pool(processes=10)
    # Result = pool.map(PC_PKM, [-29, -25, -20, -15, -10, -5, 0, 5, 8])
    Result = pool.map(PC_PKM, [-29,  -10,  0, 8])
    pool.close()
    pool.join()
    df = pd.DataFrame(Result)
    df = df
    nametable=[]
    # создаются новые таблицы после каждого расчета
    for i in range(len(Result)):
        nametable = "resdataTemp"+str(df['T_air'][i])+".xlsx"
        with pd.ExcelWriter(nametable) as writer:
            df.to_excel(writer, sheet_name="all_result", index=False)
            
